# MLE7 - Spark

1. Background

The purpose of this exercise is to teach you to use Spark for conducting distributed machine learning computation.

2. The task

Your task is the Bosch challenge (https://www.kaggle.com/c/bosch-production-line-performance) from Kaggle. This is a binary classification task, with a rather imbalanced response variable.

You can fetch the data either from the Kaggle site, or from an S3 bucket (s3://mle7-data, available from within sandbox AWS account). The response variable is included in the file train_numerical.csv. Feel free to start with only numerical variables, and include categorical and date variables later.

Notice that the test data does not include the response variable. If you wish to evaluate your model, you can do it by splitting your training data into test and train subsets.

Do not spent too much time on model improvement and evaluation -- it is more important that you learn to do machine learning with Spark.

In [4]:
import pyspark
from pyspark.sql import SparkSession
from pyspark import SparkContext
from pyspark import SparkConf
from pyspark.sql import SQLContext
from pyspark.ml.feature import VectorAssembler
from pyspark.ml import Pipeline
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.evaluation import BinaryClassificationEvaluator
sc = pyspark.SparkContext('local[*]')
spark = SparkSession.builder.appName('mle7-matiaska').getOrCreate()

### Read data

In [6]:
df = spark.read.csv("train_numeric.csv", header="true",inferSchema="true")

In [7]:
df=df.na.fill(0)

In [9]:
columns=[x for x in df.columns if x not in ['Id', 'Response']]

In [10]:
assembler = VectorAssembler(inputCols=columns, outputCol='features')

In [11]:
df = (assembler.transform(df).select('Response',"features"))

### Split into training and testdata

In [12]:
trainingData, testData = df.randomSplit([0.8,0.2], seed=0)

### Create RandomForestClassifier

In [14]:
rf = RandomForestClassifier(labelCol='Response', featuresCol='features')

In [15]:
pipeline = Pipeline(stages=[rf])

In [17]:
model = pipeline.fit(trainingData)

In [18]:
predictions = model.transform(testData)

In [20]:
results = predictions.select(['probability', 'Response'])

### Evaluate results

In [30]:
evaluator = BinaryClassificationEvaluator().setMetricName("areaUnderROC").setRawPredictionCol("probability").setLabelCol("Response")
aucTraining = evaluator.evaluate(dataset=results)

In [31]:
aucTraining

0.6843998496574835